# Moduły, pakiety i paczkowanie oprogramowania

## System importów
Moduły biblioteczene stają się dostępne, gdy się je zaimportuje:

In [ ]:
import functools
functools.reduce(lambda x, y: x + y, [1,2,3,4,5,6,7,8,9,10], 100)

Można importować całe moduły i posługiwać się nimi prefiksując nazwy w nich zawarte nazwą modułu (jak wyżej), lub importować te nazwy do głównej przestrzeni nazw selektywnie - należy wówczas uważać na potencjalne konflikty!

In [ ]:
from functools import reduce
reduce(lambda x, y: x + y, [1,2,3,4,5,6,7,8,9,10], 100)

Modułom można nadawać aliasy - przydatne np.: gdy chcemy unikać konfliktu nazw i pisać trochę mniej znaków:

In [ ]:
import functools as ftools
ftools.reduce(lambda x, y: x + y, [1,2,3,4,5,6,7,8,9,10], 100)

## Co się dzieje, gdy Python interpretuje `import mod`
1. Najpierw szuka pliku `mod.py` w katalogu z którego uruchomiony był skrypt lub aktualny katalog z którego uruchomiony został tryb interaktywny
2. Następnie jeśli ustawiona jest zmienna `$PYTHONPATH` to po kolei przeszukiwane są kolejne katalogi tam wymienione (w fomacie analogicznym jak `$PATH`)
3. Na końcu przeszukiwane są katalogi systemowe, wybrane podczas instalacji Pythona

W razie wątpliwości, można przyjrzeć się zmiennej `path` z modułu `sys`:

In [ ]:
import sys
sys.path

W związku z tym, gdy zrobimy własny moduł `my_mod`, należy go umieścić w jednym z w/w miejsc, a jeśli to konieczne zmodyfikować `$PYTHONPATH` (z powłoki, z której odpalamy Pythona lub programatycznie przez `sys.path.append(...)`), tak by zawierał katalog, w którym znajduje się plik `my_mod.py`

Każdy moduł ma swoją tablicę symboli, więc samo użycie `import mod` nie oznacza dodania występujących tam nazw do aktualnego namespace'a. Aby zobaczyć lokalną tablicę symboli i zrozumieć co dzieje się z nią w przypadku importowania modułów na różne sposoby, można wykorzystać funkcję `dir`:

In [ ]:
import abc
dir(abc)

## Moduły
Każdy plik Pythona o rozszerzeniu `*.py`, zawierający kod jest modułem, który można zazwyczaj zaimportować używając nazwy pliku - np.: mając swój kod w pliku `bulbulator.py`, importujemy w innym pliku zdefiniowane w nim nazwy przy użyciu `import bulbulator`. Wszelkie definicje i kod wykonywalny zostaną zinterpretowane raz - w momencie, gdy moduł jest importowany. Jeśli chcemy wykonać kod zawarty w module jeszcze raz, mamy do dyspozycji bibliotekę `importlib`:

In [ ]:
import importlib

importlib.reload(sys)

Moduły posiadają specjalną zmienną zmienną `__name__`, przechowującą ich nazwę:

In [ ]:
import itertools
itertools.__name__

Możliwe jest też wykonanie modułu jako skryptu - wystarczy wykonać polecenie:

```
python3 mój_moduł.py
```

Wówczas wszystkie znajdujące się na najwyższym poziomie zmienne i wywołania funkcji zostaną wyliczone, a zmienna `__name__` przybierze specjalną wartość `__main__`. Ten ostatni fakt jest wykorzystywany gdy tworzymy aplikację mającą określony punkt wejśćia - zwykle zgodnie z konwencją plik zawierający Pythonowy odpowiednik funkcji `main` znanej np.: z C i Javy zawiera następującą konstrukcję:

```

def main():
    ...

if __name__ == "__main__":
    main()
```

#### Argumenty z linii komend
W przypadku gdy skrypt bierze dodatkowe argumenty są one dostępne przez specjalną zmienną `argv` modułu sys:

In [ ]:
import sys

def main():
    for arg in sys.argv:
        print(arg)

if __name__ == "__main__":
    main()

## Pakiety
Pakiety (ang. *packages*) w Pythonie to po prostu katalogi zawierające moduły Pythona. Aby zaimportować moduły `mod1` i `mod2` z katalogu pkg możemy użyć jednego z poniższych sposobów:

In [ ]:
from pkg import mod1, mod2

mod1.foo()

In [ ]:
import pkg.mod1, pkg.mod2
pkg.mod1.foo()

In [ ]:
import pkg.mod1 as mod1
import pkg.mod2 as mod2
mod1.foo()

In [ ]:
from pkg import mod1 as mod1_alias
from pkg import mod2 as mod2_alias

mod1_alias.foo()

In [ ]:
from pkg.mod1 import foo

foo()

Można też użyć po prostu `import pkg`, jednak domyślnie ta opcja nie wprowadza do tablicy symboli żadnych nowych wpisów, chyba, że żądane moduły są explicite wymienione w pliku `__init__.py`.

#### `__init__.py`
Historycznie aby katalog był poprawnie rozpoznany jako pakiet musiał znajdować się w nim plik `__init__.py` - nawet pusty. Od czasu Pythona 3.3 tego wymagania nie ma (patrz [PEP-420](https://www.python.org/dev/peps/pep-0420/)), jednak jest to dobra praktyka w sytuacji gdy chcemy mieć kontrolę nad tym co dany pakiet eksportuje i co się dzieje gdy ktoś importuje moduły za pomocą `from pkg import *`. Wówczas w tym pliku umieszcza się listę `__all__` zawierającą wszystkie eksportowane nazwy:

In [ ]:
__all__ = [
        'mod1',
        'mod2',
        'mod3',
        'mod4'
        ]

Pakiety mogą mieć subpakiety:

```
pkg/
|
|->sub_pkg1/
|   |
|   |->mod1.py
|
|->sub_pkg2
    |
    |->mod2.py
```

In [ ]:
import pkg.sub_pkg1.mod1
pkg.sub_pkg.mod.foo()

In [ ]:
from pkg.sub_pkg1 import mod1
mod.foo()

In [ ]:
from pkg.sub_pkg1.mod1 import foo

foo()

Importy mogą być też relatywne:

In [ ]:
from ..sub_pkg2 import mod2

## Instalacja i używanie zewnętrznych bibliotek
Wokół Pythona istnieje bogaty ekosystem bibliotek i frameworków, które są dostępne w centralnym repozytorium pakietów pod adresem `http://pypi.org`. Aby zainstalować odpowiednie pakiety, można skorzystać z narzędzia `pip`, np.:
```
pip install flask
```

pip automatycznie ściągnie i rozpakuje paczkę z pypi, a nastepnie umieści ją albo w systemowym katalogu (można go znaleźć w `$PYTHONPATH` przy użyciu `python -c "import sys; print(sys.path)"` albo w wirtualnym środowisku (utworzonym albo `virtualenv`, albo modułem `venv`).

### `requirements.txt`
Standardowo, aplikacje pythonowe dystrybuuje się wraz z plikiem zawierającym listę zależności i ich wersji, wymaganych przez aplikację. Ma ona następujący format:

```
<nazwa pakietu>==<wersja>
```

np.:
```
flask>=2.0.1
```

Możliwe jest używanie znaków `<=`, `>=` etc. jeśli nie zależy nam na konkrentnej wersji, a jedynie np.: wersji nie starszej niż- lub nie nowszej niż- określona. Dobrą praktyką jest jednak przypinanie konkretnej wersji i używanie pip-tools do generowania requirements.txt.

Instalacja pakietów z listy możliwa jest poleceniem:
```
pip install -r requirements.txt
```

### `requirements.in` i `pip-tools`
Przypięte na stałe wersje pakietów są dobre z punktu widzenia powtarzalności konfiguracji - mamy pewność, że nasza aplikacja będzie używała tych samych paczek lokalnie, w CI i na produkcji. Idą za tym jednak pewne minusy - a konkretniej konieczność ręcznego znajdowania współpracujących wersji pakietów i niewygodne podbijanie ich do najnowszej wersji. Pomóc może uzycie `pip-tools`: zamiast gotowego pliku `requirements.txt`, możemy użyć pliku `requirements.in`, zawierającego listę pakietów bez przypiętych wersji (lub spełniających dużo luźniejsze ograniczenia). Następnie generujemy docelowy plik `requirements.txt` poleceniem:
```
pip-compile requirements.in --output-file requirements.txt
```

`pip-compile` automatycznie znajdzie najnowsze współgrające ze sobą wersje pakietów na `pypi`, a następnie zapisze je w `requirements.txt`. Od teraz, jeśli `requirements.txt` jest generowane lokalnie na maszynie developera, mamy wszystkie benefity zwiazane z powtarzalnością konfiguracji, jak również prostotę znajdowania odpowiednich wersji i ich aktualizacji. Aby podbić wersje pakietów można skorzystać z polecenia:

```
pip-compile --upgrade --output-file requirements.txt
```

Aby zainstalować znalezione i zahardkodowane w `requirements.txt` pakiety można użyć albo `pip install -r requirements.txt` albo `pip-sync requirements.txt`. W ten sposób otrzymujemy wygodny sposób zarządzania zależnościami naszej aplikacji - należy jednak przetestować działanie aplikacji z nowymi wersjami zależności!

### *Zadanie*
W sklonowanym repozytorium z zadaniem wykonaj: 
```
git checkout task-0
git checkout -b my-solution-0
pip install virtualenv
virtualenv <nazwa katalogu>
source env/bin/activate # Linux
.\env\Scripts\activate # Windows

alternatywnie
python3.9 -m venv <nazwa katalogu>
source env/bin/activate # Linux
.\env\Scripts\activate # Windows
```

a następnie:
- [ ] zainstaluj pip-tools (`python -m pip install pip-tools`)
- [ ] stwórz plik `requirements.in` zawierający w osobnych liniach `flask`, `click`, oraz `pytest`
- [ ] uruchom `pip-compile requirements.in --output-file=requirements.txt` aby przypiać wersje pakietów
- [ ] zainstaluj pakiety poleceniem `pip-sync`
- [ ] scommituj oba pliki `requirements.*` na utworzonym przez siebie branchu repozytorium gitowego 

[SPOILER]: Aby podejrzeć stan repozytorium po wykonaniu ćwiczenia wykonaj polecenie `git checkout solution-0` - zauważ, że nie commituje się `virtualenv`'ów, tylko same pliki `requirements.*`!